In [ ]:
%load_ext autoreload
%autoreload 2

from pim.models.network import ForwardNetwork, RecurrentNetwork, InputLayer, FunctionLayer, IdentityLayer

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x)) - 0.5

def relu(x):
    return np.fmax(np.array([0.0]), x[0])

def hidden(inputs):
    x, y = inputs
    print(y)
    return sigmoid(x) + 0.6 * y

In [ ]:
T = np.linspace(-10, 10, 100)

In [ ]:
plt.plot(T, sigmoid([T]))

In [ ]:
input_layer = InputLayer()
network = RecurrentNetwork({
    "in": input_layer,
    "hidden": FunctionLayer(inputs = ["in"], function = lambda x: sigmoid(x[0])),
    "integrator": FunctionLayer(inputs = ["hidden", "integrator"], function = lambda x: x[0]+ 0.5 + x[1]),
    "out": FunctionLayer(inputs = ["integrator"], function = relu),
})

Y = network.simulate(input_layer, "integrator", T)

plt.plot(T, Y)

In [ ]:
import networkx as nx
nx.draw(network.get_graph(), with_labels=True)